In [108]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from vrp import VRP

In [109]:
class MetaEnv(gym.Env):
    def __init__(self, vrp):
        self.observation_space = spaces.Dict(
            {
                "objective": gym.spaces.Box(low=-1000.0, high=1000.0, shape=(1,), dtype=np.float32),
                "mutation_rate": gym.spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32),
                "current_iteration": gym.spaces.Box(low=0, high=100000, shape=(1,), dtype=int)
            }
        )
        self.action_space = spaces.Dict(
            {
                "mode": spaces.Discrete(3),
            }
        )
        
        self.vrp = vrp

    def _get_obs(self):
        """Convert internal state to observation format.

        Returns:
            dict: Observation
        """
        return {
            "objective" : self.vrp.get_best_solution(),
            "mutation_rate": self.vrp.F,
            "current_iteration": self.vrp.current_iteration,
        }

    def _get_info(self):
        """Compute auxiliary information for debugging.

        Returns:
            dict: Info with distance between agent and target
        """
        return {
            "delta_F": self.vrp.delta_F
        }

    def reset(self, seed=None):
        self.vrp.reset()
        super().reset(seed=seed)
        observation = self._get_obs()
        info = self._get_info()
        return observation, info

    def step(self, action):
        
        if action == 1:
            self.vrp.change_F("DECREASE")
        elif action == 2:
            self.vrp.change_F("INCREASE")
            
        self.vrp.evolve(n_iteration=100)
        
        reward = 10
        if self.vrp.is_exceed_max_iteration():
            terminated = True
            truncated = True
        else:
            terminated = False
            truncated = False
        observation = self._get_obs()
        info = self._get_info()
        return observation, reward, terminated, truncated, info

In [110]:
distance = np.array(
    [
        [0, 0.664, 1.035, 1.789, 4.854, 7.586, 11.425, 11.871],
        [4.857, 0, 0.852, 1.606, 4.671, 7.403, 11.242, 11.688],
        [4.004, 4.23, 0, 0.753, 3.818, 6.55, 10.389, 10.835],
        [5.857, 6.083, 6.454, 0, 3.064, 5.796, 9.635, 10.081],
        [7.267, 7.493, 7.864, 4.066, 0, 2.934, 6.698, 7.144],
        [10.704, 10.93, 11.301, 7.503, 3.68, 0, 5.619, 6.065],
        [13.475, 13.701, 14.072, 10.274, 6.451, 5.409, 0, 0.943],
        [15.079, 15.305, 15.676, 11.878, 8.055, 7.013, 1.603, 0],
    ]
)


dimensions = len(distance) - 1
maxiters = 100
population_size = 100
bounds = np.array([[0, 1]] * dimensions)
Mutation_rate = np.array([0.5, 0.9])
Crossover_rate = np.array([0.1, 0.5])


vrp = VRP(
    population_size=population_size,
    dimensions=dimensions,
    bounds=bounds,
    Mutation_rate=Mutation_rate,
    Crossover_rate=Crossover_rate,
    distance=distance,
    max_iteration=1000
)

In [111]:
from gymnasium.utils.env_checker import check_env

env = MetaEnv(vrp=vrp)

# This will catch many common issues
try:
    check_env(env)
    print("Environment passes all checks!")
except Exception as e:
    print(f"Environment has issues: {e}")

Environment has issues: The first element returned by `env.reset()` is not within the observation space.


In [131]:
env = MetaEnv(vrp=vrp)

print(env.reset())
print(env.action_space.sample())

({'objective': 0.7948087147490299, 'mutation_rate': np.float64(0.5), 'current_iteration': 0}, {'delta_F': np.float64(0.004)})
{'mode': np.int64(2)}


In [150]:
env.step(action=2)

({'objective': 0.3030798313348515,
  'mutation_rate': np.float64(0.5),
  'current_iteration': 1900},
 10,
 True,
 True,
 {'delta_F': np.float64(0.004)})

In [114]:
# env = MetaEnv(vrp=vrp)

# env.reset()
# done = False
# count = 0
# while not done:
#     env.step()
#     count = count + 1
#     if count > 10:
#         done = True
    
